In [9]:
import os
import xml.etree.ElementTree as ET
from datetime import datetime
from pymongo import MongoClient

In [10]:
def read_all_file_names(directory_path):
    file_names = []
    try:
        # List all items in the directory
        for file_name in os.listdir(directory_path):
            file_path = os.path.join(directory_path, file_name)
            # Check if it's a file
            if os.path.isfile(file_path):
                file_names.append(file_name)
    except Exception as e:
        print(f"Error reading file names in directory: {e}")
    return file_names

# Example usage
# directory_path = r"C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable"
directory_path = r"C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone"
file_names = read_all_file_names(directory_path)
# log_file = "analyzed-manifest-remain.log"
log_file = "analyzed-manifest-standalone.log"
# print(file_names)
print(len(file_names))

73


In [11]:
def write_log(log_file, log_text):
    try:
        # Get the current timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # Write the log message to the file
        with open(log_file, 'a') as file:
            file.write(f"[{timestamp}] {log_text}\n")
    except Exception as e:
        print(f"Failed to write log: {e}")

In [12]:
def extract_meta_data_tags(file_path, tag_name):
    try:
        # Read the XML file content
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_content = file.read()

        # Parse the XML content
        root = ET.fromstring(xml_content)

        # Search for the specified tag
        tags = root.findall(f".//{tag_name}")

        if not tags:
            print(f"No <{tag_name}> tags found in the XML content.")
            return []

        # Extract and return the attributes of each found tag
        tag_list = [tag.attrib for tag in tags]
        return tag_list
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

In [13]:
mongoDB_uri = 'mongodb://localhost:27017'
mongoDB_database = 'wearable-project' 
# mongoDB_collection = 'wearable-app'
mongoDB_collection = 'wearable-standalone'
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
collection = connect_mongodb(mongoDB_uri, mongoDB_database, mongoDB_collection)

In [14]:
# Function to update an existing document with a new key and value
def add_tag_list_to_document(collection, document_id, tag_key, tag_list):
    try:
        # Update the document by adding a new key-value pair
        update_result = collection.update_one(
            {"_id": document_id},  # Find the document with the specified _id
            {"$set": {tag_key: tag_list}}  # Set the new key-value pair
        )

        # Check if the update was successful
        if update_result.matched_count > 0:
            print(f"Successfully updated document with _id: {document_id}")
        else:
            print(f"No document found with _id: {document_id}")
    except Exception as e:
        print(f"Error updating document: {e}")


In [15]:
def get_id_from_filename(filename):
    return filename.split('-')[0]

In [16]:
# manifest_tag_arr = ["action","application","data","property","provider",
#                     "receiver","service","uses-configuration","uses-feature","uses-library"]
#manifest_tag_arr = ["meta-data","property","uses-configuration"]
manifest_tag_arr = ["meta-data"]
for j in range(len(manifest_tag_arr)):
    tag_name = manifest_tag_arr[j]
    print("=-=-=-=-=-=-=- Process tag name "+str(j) +": "+tag_name+" =-=-=-=-=-=-=-")
    for i in range(len(file_names)):
        file_name = file_names[i]
        document_id = get_id_from_filename(file_name)
        print("--------- Process file number "+str(i) +": "+file_name+" ---------")
        xml_file_path = directory_path + "\\" + file_name
        print("file_path",xml_file_path)
        write_log(log_file, "--------- Process file number "+str(i) +": "+xml_file_path+" ---------")
#         tag_name = "property"  
        found_tags = extract_meta_data_tags(xml_file_path, tag_name)
        print(found_tags)
        if found_tags:
            print(f"Found <{tag_name}> tags:")
            write_log(log_file, f"Found <{tag_name}> tags")
            print(type(found_tags))
            write_log(log_file, found_tags)
            tag_key = "tag-"+tag_name  # The key to add to the document
            add_tag_list_to_document(collection, document_id, tag_key, found_tags)
        else:
            print(f"Not found <{tag_name}>.")
            write_log(log_file, f"Not found <{tag_name}>.")
            add_tag_list_to_document(collection, document_id, tag_key, "not-found")
    break
#         break

=-=-=-=-=-=-=- Process tag name 0: meta-data =-=-=-=-=-=-=-
--------- Process file number 0: app.groupcal.www-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\app.groupcal.www-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'screenCaptureProcessors', '{http://schemas.android.com/apk/res/android}value': 'com.google.firebase.testlab.screenshot.FirebaseScreenCaptureProcessor'}, {'{http://schemas.android.com/apk/res/android}name': 'CLEVERTAP_ACCOUNT_ID', '{http://schemas.android.com/apk/res/android}value': 'R7R-47Z-K55Z'}, {'{http://schemas.android.com/apk/res/android}name': 'CLEVERTAP_TOKEN', '{http://schemas.android.com/apk/res/android}value': '1a4-2a2'}, {'{http://schemas.android.com/apk/res/android}name': 'CLEVERTAP_USE_GOOGLE_AD_ID', '{http://schemas.android.com/apk/res/android}value': '1'}, {'{http://schemas.android.com/apk/res/android}name': 'com.samsung.android.icon_container.has_icon_contai

<class 'list'>
Successfully updated document with _id: com.fitbod.fitbod
--------- Process file number 16: com.fitiv.fitivapplication-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\com.fitiv.fitivapplication-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'AccessoryServicesLocation', '{http://schemas.android.com/apk/res/android}value': '/res/xml/accessoryservices.xml'}, {'{http://schemas.android.com/apk/res/android}name': 'com.facebook.sdk.ApplicationId', '{http://schemas.android.com/apk/res/android}value': '@7F140CF2'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.geo.API_KEY', '{http://schemas.android.com/apk/res/android}value': '@7F140D05'}, {'{http://schemas.android.com/apk/res/android}name': 'com.facebook.sdk.ClientToken', '{http://schemas.android.com/apk/res/android}value': '@7F140CF3'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.firebase

[{'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.version', '{http://schemas.android.com/apk/res/android}value': '@7F090008'}, {'{http://schemas.android.com/apk/res/android}name': 'androidx.emoji2.text.EmojiCompatInitializer', '{http://schemas.android.com/apk/res/android}value': 'androidx.startup'}, {'{http://schemas.android.com/apk/res/android}name': 'androidx.lifecycle.ProcessLifecycleInitializer', '{http://schemas.android.com/apk/res/android}value': 'androidx.startup'}, {'{http://schemas.android.com/apk/res/android}name': 'androidx.profileinstaller.ProfileInstallerInitializer', '{http://schemas.android.com/apk/res/android}value': 'androidx.startup'}, {'{http://schemas.android.com/apk/res/android}name': 'com.android.dynamic.apk.fused.modules', '{http://schemas.android.com/apk/res/android}value': 'base'}, {'{http://schemas.android.com/apk/res/android}name': 'com.android.stamp.source', '{http://schemas.android.com/apk/res/android}value': 'https://play.google

<class 'list'>
Successfully updated document with _id: com.juventus.app.android
--------- Process file number 38: com.krisdb.wearcasts-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\com.krisdb.wearcasts-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'android.support.FILE_PROVIDER_PATHS', '{http://schemas.android.com/apk/res/android}resource': '@7F160007'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.version', '{http://schemas.android.com/apk/res/android}value': '@7F0B002C'}, {'{http://schemas.android.com/apk/res/android}name': 'androidx.car.app.TintableAttributionIcon', '{http://schemas.android.com/apk/res/android}resource': '@7F080167'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.car.notification.SmallIcon', '{http://schemas.android.com/apk/res/android}resource': '@7F080131'}, {'{http://schemas.android.com/apk/res/android}nam

<class 'list'>
Successfully updated document with _id: com.northpark.drinkwater
--------- Process file number 47: com.padelem.padelem-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\com.padelem.padelem-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'io.flutter.embedding.android.NormalTheme', '{http://schemas.android.com/apk/res/android}resource': '@7F0F00B4'}, {'{http://schemas.android.com/apk/res/android}name': 'flutterEmbedding', '{http://schemas.android.com/apk/res/android}value': '2'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.ads.APPLICATION_ID', '{http://schemas.android.com/apk/res/android}value': 'ca-app-pub-4586285572833050~4567438574'}, {'{http://schemas.android.com/apk/res/android}name': 'com.facebook.sdk.ApplicationId', '{http://schemas.android.com/apk/res/android}value': '@7F0E004F'}, {'{http://schemas.android.com/apk/res/android}name': 'com.faceb

Successfully updated document with _id: com.vpar.android
--------- Process file number 57: com.watchfacedesigns.Vanish-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\com.watchfacedesigns.Vanish-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'io.flutter.embedding.android.NormalTheme', '{http://schemas.android.com/apk/res/android}resource': '@7F1200A9'}, {'{http://schemas.android.com/apk/res/android}name': 'flutterEmbedding', '{http://schemas.android.com/apk/res/android}value': '2'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.firebase.components:io.flutter.plugins.firebase.firestore.FlutterFirebaseFirestoreRegistrar', '{http://schemas.android.com/apk/res/android}value': 'com.google.firebase.components.ComponentRegistrar'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.firebase.components:io.flutter.plugins.firebase.messaging.FlutterFirebaseAppRegistrar',

Successfully updated document with _id: nl.slisky.stopwatch
--------- Process file number 66: qubin.de.wearmedia-AndroidManifest.xml ---------
file_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\qubin.de.wearmedia-AndroidManifest.xml
[{'{http://schemas.android.com/apk/res/android}name': 'android.app.default_searchable', '{http://schemas.android.com/apk/res/android}resource': '@7F160006'}, {'{http://schemas.android.com/apk/res/android}name': 'android.support.FILE_PROVIDER_PATHS', '{http://schemas.android.com/apk/res/android}resource': '@7F160003'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.version', '{http://schemas.android.com/apk/res/android}value': '@7F0B0012'}, {'{http://schemas.android.com/apk/res/android}name': 'com.google.android.gms.ads.APPLICATION_ID', '{http://schemas.android.com/apk/res/android}value': 'ca-app-pub-2001315674065028~6803376195'}, {'{http://schemas.android.com/apk/res/android}name': 'firebase_pe